## Creating a DataFrame form XML

In this Notebook the coordinate data that is stored in a XML is saved in a DataFrame for further use in Notebook 9.

### Import libraries

1. **re**  - This is a library to process regular expressions. 

2. **Numpy** - Numpy is a library for the easy use of vectors, matrices or arrays in general. It simplifies various numerical operations. 

3. **Codecs** - This module provides access to the most common Python encoders and decoders for example to be used for text encoding.

4. **Pandas** - Pandas is a library to analyze and to manage data. It is used to create tables.

5. **Os** - This enables a way to use the operating system.

6. Moreover, xml.etree.ElementTree and objectify from lxml are used for easier and faster processing of XML data in Python.

In [2]:
# import statements
!pip install pandas==1.3.4
import re
import numpy
import codecs
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from lxml import objectify
#import rdflib
#from rdflib.namespace import RDF, RDFS
#from rdflib import Literal
#nltk.download('punkt')
#-*- coding: utf-8 -*-
# from google.colab import drive 
# drive.mount('/content/gdrive')

  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.1
    Uninstalling pandas-1.0.1:
      Successfully uninstalled pandas-1.0.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Zugriff verweigert: 'c:\\users\\pavel nefedov\\anaconda3\\lib\\site-packages\\~andas\\_libs\\algos.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



### Import data

In [5]:
path ='./files/F_23_92.jpg/F_23_92_alto/'

files = os.listdir(path)
print(len(files))

file_path_file1 = os.path.join(path, files[0])
#file_path_file1 = '/content/gdrive/MyDrive/MA Python/F_23_92.jpg/F_23_92_alto/F_23_92-001.xml'
tree = ET.parse(file_path_file1)
root = tree.getroot()
print(root.tag, root.attrib)

for child in root:
    print(child.tag, child.attrib)

#print(ET.tostring(root, encoding='utf8').decode('utf8'))

428
{http://www.loc.gov/standards/alto/ns-v3#}alto {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.loc.gov/standards/alto/ns-v3# http://www.loc.gov/alto/v3/alto-3-0.xsd'}
{http://www.loc.gov/standards/alto/ns-v3#}Description {}
{http://www.loc.gov/standards/alto/ns-v3#}Styles {}
{http://www.loc.gov/standards/alto/ns-v3#}Layout {}


### Extract information stored in XMl into a Pandas DataFrame

In [421]:
ns = {'alto': 'http://www.loc.gov/standards/alto/ns-v3#'}
df_cols = ['Page', 'Content', 'Height', 'Width', 'VPOS', 'HPOS'] 
rows = []
for file in files:
    file_path = path+file
    #print('Processing...'+file)
    tree = ET.parse(file_path)
    root = tree.getroot()

    for Layout in root.findall('alto:Layout', ns):
        for Page in Layout.findall('alto:Page', ns):
            for PrintSpace in Page.findall('alto:PrintSpace', ns):
                for TextBlock in PrintSpace.findall('alto:TextBlock', ns):
                    for TextLine in TextBlock.findall('alto:TextLine', ns):
                            for String in TextLine.findall('alto:String', ns):
                                if String is not None:
                                    Content = String.attrib.get('CONTENT')
                                    Height = String.attrib.get('HEIGHT')
                                    Width = String.attrib.get('WIDTH')
                                    VPOS = String.attrib.get('VPOS')
                                    HPOS = String.attrib.get('HPOS')
                                else:
                                    None 
                                for Description in root.findall('alto:Description', ns):
                                    MeasurementUnit = Description.find('alto:MeasurementUnit', ns)
                                    #print(MeasurementUnit.text)
                                    for sourceImageInformation in Description.findall('alto:sourceImageInformation', ns):
                                        for fileName in sourceImageInformation.findall('alto:fileName', ns):
                                            if fileName is not None:
                                                Page = fileName.text
                                            else: 
                                                None
                                            rows.append({
                                            "Content": Content,
                                            "Height": Height,
                                            "Width": Width,
                                            "VPOS": VPOS,
                                            'HPOS' : HPOS,
                                            'Page' : Page
                                            })                             

Coordinates_df = pd.DataFrame(rows , columns = df_cols)
Coordinates_df

,Page,Content,Height,Width,VPOS,HPOS
0,F_23_92.jpg\F_23_92-001.jpg,LtacstZrcüiv,40,266,5410,1720
1,F_23_92.jpg\F_23_92-001.jpg,blümdsrg,49,217,5411,2007
2,F_23_92.jpg\F_23_92-001.jpg,-,5,15,5432,2245
3,F_23_92.jpg\F_23_92-001.jpg,öestZricI,39,190,5410,2281
4,F_23_92.jpg\F_23_92-001.jpg,1^,39,26,5410,2494
...,...,...,...,...,...,...
260625,F_23_92.jpg\F_23_92-401.jpg,"Duschner,",53,218,4687,1188
260626,F_23_92.jpg\F_23_92-401.jpg,"I.,",50,62,4687,1436
260627,F_23_92.jpg\F_23_92-401.jpg,Stanzer,54,190,4684,1529
260628,F_23_92.jpg\F_23_92-401.jpg,I,38,17,4685,1806


In [422]:
Coordinates_df.to_csv('./Outputs/Coordinates_df.csv', index=False, header=True)

In [423]:
#xml_file = '/content/gdrive/MyDrive/MA Python/F_23_92.jpg/F_23_92.jpg_alto/F_23_92-002.xml'
#prstree = ET.parse(xml_file)
#root = prstree.getroot()
#xml_df = pd.read_xml('/content/gdrive/MyDrive/MA Python/F_23_92.jpg/F_23_92.jpg_alto/F_23_92-002.xml', xpath="//alto:String", namespaces = {'alto':'http://www.loc.gov/standards/alto/ns-v3#'})
#xml_df.head(10)
